# **v3.0**


*   groqcloud API
*   Model: llama-3.3-70b-versatile
*   Has all the required agents
*   Supports dynamic agent creation
*   Supports freely evolving trial structure
*   Predicts the verdict of test_cases.csv data










### Installing groq

In [1]:
pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 kB 1.5 MB/s eta 0:00:00


## Importing Libraries

In [2]:
import pandas as pd
from groq import Groq
from typing import List, Dict, Optional
import re
import time

## Choosing Model

In [3]:
GROQ_API_KEYS = ["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"]
MODEL_NAME = "llama-3.3-70b-versatile"

## Verdict Parsing Helper

In [4]:
def parse_llm_verdict(judge_response: str):
    match = re.search(r"#VERDICT:\s*(GRANTED|DENIED)", judge_response, re.IGNORECASE)
    if match:
        return 1 if match.group(1).upper() == "GRANTED" else 0
    if "granted" in judge_response.lower() or "acquitted" in judge_response.lower():
        return 1
    return 0

# Classes Implementation

## Courtroom Agent

In [5]:
class CourtroomAgent:
    def __init__(self, name: str, role: str, system_prompt: str, client):
        self.name = name
        self.role = role
        self.system_prompt = system_prompt
        self.client = client
        self.history: List[Dict[str, str]] = [
            {"role": "system", "content": self.system_prompt}
        ]
    def add_message(self, role: str, content: str):
        self.history.append({"role": role, "content": content})
    def get_response(self, user_message: str) -> str:
        self.add_message("user", user_message)
        response = self.client.chat.completions.create(
            model=MODEL_NAME, messages=self.history, stream=False)
        reply = response.choices[0].message.content
        self.add_message("assistant", reply)
        return reply

## Courtroom

In [6]:
class Courtroom:
    def __init__(self, case_background: str, client):
        self.client = client
        self.agents: Dict[str, CourtroomAgent] = {}
        self.case_background: str = case_background
        self.case_type: str = self.determine_case_type(case_background)
        self.dynamic_agent_counter = 0
        self.add_agent("Judge", CourtroomAgent("Judge", "Judge", "You are a fair and wise judge. Weigh all arguments neutrally and deliver clear rulings.", client))
        self.add_agent("Defense", CourtroomAgent("Defense", "Defense Lawyer", "You are the defense lawyer. Protect your client and argue logically against the prosecution's claims.", client))
        self.add_agent("Defendant", CourtroomAgent("Defendant", "Defendant", "You are the defendant. Answer questions honestly and provide your perspective on the case.", client))
        self.add_agent("Prosecution", CourtroomAgent("Prosecution", "Prosecution Lawyer", "You are the prosecution lawyer. Present compelling arguments and point out flaws in the defense.", client))
        if self.case_type == "civil":
            self.add_agent("Plaintiff", CourtroomAgent("Plaintiff", "Plaintiff", "You are the plaintiff. Represent your interests and highlight injustices suffered.", client))
    def add_agent(self, key: str, agent: CourtroomAgent):
        self.agents[key] = agent
    def create_witness(self, name: Optional[str] = None, prompt: Optional[str] = None) -> str:
        self.dynamic_agent_counter += 1
        witness_name = name or f"Witness{self.dynamic_agent_counter}"
        system_prompt = prompt or f"You are {witness_name}, a witness in this case. Answer questions truthfully and relevantly."
        self.add_agent(witness_name, CourtroomAgent(witness_name, "Witness", system_prompt, self.client))
        return witness_name
    def run_phase(self, phase_name: str, prompts: Dict[str, str]) -> Dict[str, str]:
        outputs = {}
        for role, prompt in prompts.items():
            if role in self.agents:
                full_prompt = prompt + "\nCase: " + self.case_background[:4000]
                response = self.agents[role].get_response(full_prompt)
                outputs[role] = response
        return outputs
    @staticmethod
    def determine_case_type(case_background: str) -> str:
        civil_keywords = ["contract", "arbitration", "plaintiff", "commercial", "company", "tender", "writ petition", "agreement"]
        criminal_keywords = ["murder", "theft", "assault", "criminal", "prosecution", "accused", "defendant"]
        background_lower = case_background.lower()
        if any(word in background_lower for word in civil_keywords):
            return "civil"
        if any(word in background_lower for word in criminal_keywords):
            return "criminal"
        return "civil"

# Prompts for each phase

In [7]:
def get_opening_prompts(court):
    prompts = {
        "Prosecution": (
            "As the prosecution lawyer, deliver a formal opening statement. Summarize the facts of the case, outline the charges, and explain the prosecution's theory of the crime. Emphasize the seriousness of the alleged offenses and preview the evidence you intend to present."
        ),
        "Defense": (
            "As the defense lawyer, present a formal opening statement. Introduce your clients and their position, challenge the prosecution's narrative, and outline the defense's main arguments. Emphasize the presumption of innocence and the burden of proof."
        ),
        "Defendant": (
            "As the defendant, briefly introduce yourself to the court. State your relationship to the case and your initial reaction to the charges brought against you."
        ),
    }
    if court.case_type == "civil":
        prompts["Plaintiff"] = (
            "As the plaintiff, deliver a formal opening statement. Explain your grievance, the harm you have suffered, and why you are seeking relief from the court. Briefly outline the evidence and arguments you will present."
        )
    return prompts

def get_phase2_prompts(court, witness1="Witness1", expert="ExpertWitness"):
    prompts = {
        "Prosecution": (
            f"As the prosecution lawyer, conduct a direct examination of {witness1}. Ask questions to establish the facts of the case and highlight evidence that supports the prosecution's theory."
        ),
        "Defense": (
            f"As the defense lawyer, cross-examine {witness1}. Probe for inconsistencies, challenge the witness's credibility, and defend your client's position."
        ),
        expert: (
            "As the expert witness in contract law, provide your professional analysis of the arbitration clause and its legal implications in this case."
        ),
        "Defendant": (
            "As the defendant, respond to the testimonies provided by the witnesses. Clarify your actions and motivations, and address any allegations made against you."
        ),
    }
    if court.case_type == "civil":
        prompts["Plaintiff"] = (
            "As the plaintiff, respond to the testimonies and arguments presented so far. Clarify your position and highlight any evidence supporting your claims."
        )
    return prompts

def get_closing_prompts(court):
    prompts = {
        "Prosecution": (
            "As the prosecution lawyer, deliver a formal closing statement. Summarize the prosecution's case, review the key evidence and testimonies, and argue why the defendant should be found guilty beyond a reasonable doubt."
        ),
        "Defense": (
            "As the defense lawyer, deliver a formal closing statement. Summarize your defense, highlight weaknesses in the prosecution's case, and argue for your client's acquittal."
        ),
        "Defendant": (
            "As the defendant, present your final remarks to the court. Express your perspective on the trial and reiterate your innocence or mitigating circumstances."
        ),
    }
    if court.case_type == "civil":
        prompts["Plaintiff"] = (
            "As the plaintiff, deliver your closing statement. Summarize your case, the harm suffered, and why the court should rule in your favor."
        )
    return prompts

def get_judge_ruling_prompt():
    return {
        "Judge": (
            "As the judge, review the arguments, evidence, and testimonies presented during the trial. "
            "Deliver your verdict with clear legal reasoning, referencing the facts and applicable law. "
            "At the END of your response, write ONLY one of these tags on a new line:\n"
            "#VERDICT: GRANTED   (if the relief/petition/appeal should be granted or the defendant is acquitted)\n"
            "#VERDICT: DENIED    (if the relief/petition/appeal should be denied or the defendant is convicted)\n"
            "Do NOT output anything else on the line with the tag."
        )
    }


# Getting the final verdict

In [9]:
def run_full_trial_and_get_verdict(case_text, client):
    court = Courtroom(case_background=case_text, client=client)
    witness1 = court.create_witness(name="Witness1")
    expert = court.create_witness(name="ExpertWitness")
    court.run_phase("Opening Statements", get_opening_prompts(court))
    court.run_phase("Witness Interrogation & Argumentation", get_phase2_prompts(court, witness1, "ExpertWitness"))
    court.run_phase("Closing Statements", get_closing_prompts(court))
    judge_response = court.run_phase("Judge's Ruling", get_judge_ruling_prompt())["Judge"]
    return judge_response

def get_reset_seconds_from_error_msg(error_msg):
    import re
    match = re.search(r'try again in ([\d\.]+)s', error_msg)
    if match:
        return int(float(match.group(1))) + 1  # Add a buffer
    return 900  # Default to 15 min

def batch_predict_no_csv(input_csv="test_cases.csv", max_cases=50):
    df = pd.read_csv(input_csv)
    key_idx = 0
    n_keys = len(GROQ_API_KEYS)
    for idx, row in df.head(max_cases).iterrows():
        case_id = row['id'] if 'id' in row else row['ID']
        case_text = row.get('text') or row.get('case_text') or row[1]
        attempts = 0
        while True:
            api_key = GROQ_API_KEYS[key_idx]
            client = Groq(api_key=api_key)
            try:
                judge_response = run_full_trial_and_get_verdict(case_text, client)
                verdict = parse_llm_verdict(judge_response)
                print(f"{case_id},{verdict}")
                break  # success!
            except Exception as e:
                msg = str(e)
                if "rate limit" in msg or "429" in msg:
                    print(f"# API key {key_idx+1} out of tokens or rate-limited.")
                    key_idx = (key_idx + 1) % n_keys
                    attempts += 1
                    if attempts >= n_keys:
                        # All keys exhausted, so wait!
                        wait_time = get_reset_seconds_from_error_msg(msg)
                        print(f"# All keys exhausted. Waiting {wait_time//60} min {wait_time%60}s before retrying case {case_id} ...")
                        time.sleep(wait_time)
                        attempts = 0
                else:
                    print(f"{case_id},0")
                    print(f"# Case {case_id} failed: {e}")
                    break

if __name__ == "__main__":
    batch_predict_no_csv("test_cases.csv", max_cases=50)

1989_75,1


KeyboardInterrupt: 